This notebook is used to test the customized pytensor OPs used for stop-respond and successful inhibition likelihood functions.

# Load modules, simulated data, and true parameters

In [1]:
from util import *
import pandas as pd
import pytensor

In [2]:
# Load data and test parameters
simulated_data = pd.read_csv('../simulation/simulated_data/hierarchical_simulated_data_fixed_SSD.csv')
true_parameters = pd.read_csv('../simulation/true_param/hierarchical_true_parameters_fixed_SSD.csv')

In [3]:
# Determine uppper bound of observed sst
par_0_df = simulated_data[simulated_data['participant_id'] == 0]
upper_bound = max(par_0_df['observed_rt']) + 100
print(upper_bound)

932.0086387647641


In [4]:
ssd_array_stop_respond = par_0_df.loc[par_0_df['outcome'] == 'stop-respond', 'ssd'].values
print(ssd_array_stop_respond)

[480. 400. 320.  80. 400.  80.]


In [5]:
t_r = par_0_df.loc[par_0_df['outcome'] == 'stop-respond', 'observed_rt'].values
print(t_r)

[642.02442528 629.361257   474.4613705  511.31397965 505.62656202
 544.48497631]


In [6]:
ssd_array_succcessful_inhibition = par_0_df.loc[par_0_df['outcome'] == 'successful inhibition', 'ssd'].values
print(ssd_array_succcessful_inhibition)

[320. 160. 160. 160.]


# Test Stop_Respond OP

In [7]:
it_r, (imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf), issd_array = t_r, list(true_parameters.loc[0][1:]), ssd_array_stop_respond

In [8]:
# create our Op
loglike_op = Stop_Respond()

test_out = loglike_op(it_r, imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf, issd_array)

In [9]:
pytensor.dprint(test_out, print_type=True)

Stop_Respond [id A] <Vector(float64, shape=(6,))>
 ├─ [642.02442 ... .48497631] [id B] <Vector(float64, shape=(6,))>
 ├─ 492.7769710466497 [id C] <Scalar(float64, shape=())>
 ├─ 67.98710015313769 [id D] <Scalar(float64, shape=())>
 ├─ 35.06940670098028 [id E] <Scalar(float64, shape=())>
 ├─ 160.16246583034786 [id F] <Scalar(float64, shape=())>
 ├─ 1.0000000530603947 [id G] <Scalar(float64, shape=())>
 ├─ 1.7642111746992046 [id H] <Scalar(float64, shape=())>
 ├─ 0.6595437195444122 [id I] <Scalar(float64, shape=())>
 └─ [480. 400. ... 400.  80.] [id J] <Vector(float64, shape=(6,))>


In [10]:
test_out.eval()

array([-6.67112924, -6.62764453, -5.4520588 , -5.66246806, -5.26119387,
       -5.69425052])

In [11]:
def stop_respond_likelihood(t_r, mu_go, sigma_go, tau_go, 
                            mu_stop, sigma_stop, tau_stop, 
                            p_tf, ssd):
    
    failed_trigger = p_tf * exgaussian_pdf_numpy(t_r, mu_go, sigma_go, tau_go)
    # Focus on the relevant time interval for determining if the stop process 
    # finished before the response was made (specifically, t_r - ssd_array)
    successful_trigger = (1 - p_tf) * (1 - exgaussian_cdf_numpy(t_r - ssd, mu_stop, sigma_stop, tau_stop)) * exgaussian_pdf_numpy(t_r, mu_go, sigma_go, tau_go)
    
    return np.log(failed_trigger + successful_trigger)

In [12]:
print(test_out.eval() == stop_respond_likelihood(it_r, imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf, issd_array))

[ True  True  True  True  True  True]


# Test Successful_Inhibition OP

In [13]:
iupper_bound, (imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf), issd_array = upper_bound, list(true_parameters.loc[0][1:]), ssd_array_succcessful_inhibition

In [14]:
# create our Op
loglike_op = Succcessful_Inhibition()

test_out = loglike_op(iupper_bound, imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf, issd_array)

In [15]:
pytensor.dprint(test_out, print_type=True)

Succcessful_Inhibition [id A] <Vector(float64, shape=(4,))>
 ├─ 932.0086387647641 [id B] <Scalar(float64, shape=())>
 ├─ 492.7769710466497 [id C] <Scalar(float64, shape=())>
 ├─ 67.98710015313769 [id D] <Scalar(float64, shape=())>
 ├─ 35.06940670098028 [id E] <Scalar(float64, shape=())>
 ├─ 160.16246583034786 [id F] <Scalar(float64, shape=())>
 ├─ 1.0000000530603947 [id G] <Scalar(float64, shape=())>
 ├─ 1.7642111746992046 [id H] <Scalar(float64, shape=())>
 ├─ 0.6595437195444122 [id I] <Scalar(float64, shape=())>
 └─ [320. 160. ... 160. 160.] [id J] <Vector(float64, shape=(4,))>


In [16]:
test_out.eval()

[0.07521771 0.02941847 0.02941847 0.02941847]
The likelihood is:  [0.02560834 0.0100157  0.0100157  0.0100157 ]


array([-3.66483712, -4.60360108, -4.60360108, -4.60360108])

In [17]:
def successful_inhibit_log_likelihood(mu_go, sigma_go, tau_go, 
                                      mu_stop, sigma_stop, tau_stop, 
                                      p_tf, upper_bound, ssd, n=50):

    total_integral = 0
    nodes, weights = legendre_quadrature(upper_bound, ssd, n)

    for j in range(n):
        integrand = (1 - exgaussian_cdf_numpy(nodes[j], mu_go, sigma_go, tau_go)) * exgaussian_pdf_numpy(nodes[j] - ssd, mu_stop, sigma_stop, tau_stop)
        integral = weights[j] * integrand
        total_integral += integral

    likelihood = (1 - p_tf) * total_integral

    return np.log(likelihood)

In [18]:
print(test_out.eval() == successful_inhibit_log_likelihood(imu_go, isigma_go, itau_go, imu_stop, isigma_stop, itau_stop, ip_tf, np.array([iupper_bound]), issd_array))

[ True  True  True  True]
